In [1]:
#Let's try to make these calls faster
import tekore as tk
import asyncio
import yaml
import requests


In [2]:
credentials_path = "credentials.yaml"
with open(credentials_path) as file:
    cred = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
conf = cred['client_id'], cred['client_secret']

app_token = tk.request_client_token(*conf)

#Let's try to connect these words
spotify = tk.Spotify(app_token)


In [4]:
app_token.access_token

'BQAKfG-zUZ5_wOWADgTXc9lcoMcDOUL9OWB8Ek2mpL2ES3CtGTsu1OwW-034DUxh0XbYsdsoCULPfi31zH4'

## Playlist search

In [8]:
obj = spotify.search("Top 50 USA", types=("playlist",), limit=1)

In [9]:
ids = []
for track in spotify.playlist_items(obj[0].items[0].id, as_tracks=True)['items']:
    ids.append(track["track"]["id"])


In [10]:
features = spotify.tracks_audio_features(ids)

In [11]:
type(features)

tekore.model.ModelList

## Track search

In [9]:
obj = spotify.search("Show must go on", types=("track",), limit=1)

In [10]:
obj[0].items[0].id

'3m86qx7aIeqFcYcQ1UXUvP'

In [11]:
features = spotify.track_audio_features(obj[0].items[0].id)

In [107]:
features

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

In [28]:
spotify.track_audio_features('3m86qx7aIeqFcYcQ1UXUvP')

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

### Dummy dataset

In [4]:
import sys
sys.path.append("app")
import pandas as pd
%load_ext autoreload

%autoreload 2

from spotifyData import getdata_faster
from utils import request_manager
creator = getdata_faster.DatasetCreator(spotify_api=spotify)

request_manager = request_manager.RequestManager()

In [15]:
dfs = []
#phrases = ['Top 50 Australia', 'Top 50 Poland', 'Top 50 USA']
#phrases = ['Funky 80\'s', 'best metal songs', 'best rock']
phrases = ["Happy mix", "best sad songs"]
for phrase in phrases:
    outcome = creator.search(phrase, type="playlist")
    rows = request_manager.add_data([], outcome, query_name=phrase)
    request_manager.add_request(phrase, "playlist")
    dfs.append(pd.DataFrame(rows))


<class 'tekore.model.ModelList'>
<class 'tekore.model.ModelList'>


In [16]:
df = pd.concat(dfs, axis=0)

In [17]:
df.to_csv("Different_modes.csv")

In [18]:
df

,artist,explicit,name,album,popularity,acousticness,danceability,duration,energy,instrumentalness,...,liveness,loudness,mode,speechiness,time_signature,valence,tempo,id,query,count
0,Joni Mitchell,False,Big Yellow Taxi,Ladies of the Canyon,24,0.5790,0.611,2,0.470,0.000000,...,0.5810,-9.135,major,0.0356,4,0.970,85.527,10,Happy mix,1
1,The Rascals,False,A Beautiful Morning - Single Version,Time Peace: The Rascals' Greatest Hits,52,0.3990,0.503,2,0.484,0.000000,...,0.3450,-12.833,minor,0.0311,4,0.800,122.203,10,Happy mix,1
2,The Foundations,False,Build Me Up Buttercup - Mono,The Foundations,73,0.3150,0.659,2,0.513,0.000000,...,0.2830,-7.186,major,0.0293,4,0.855,133.609,10,Happy mix,1
3,Jeff Lynne,False,Every Little Thing,Armchair Theatre,36,0.2670,0.576,3,0.560,0.000037,...,0.2640,-7.296,major,0.0362,4,0.724,115.326,10,Happy mix,1
4,George Harrison,False,Got My Mind Set On You - Remastered 2004,Cloud Nine,71,0.4650,0.767,3,0.854,0.000002,...,0.4850,-6.947,major,0.0426,4,0.963,149.206,10,Happy mix,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,James Bay,False,Let It Go,Chaos And The Calm,71,0.8200,0.536,4,0.311,0.000017,...,0.1070,-10.396,major,0.0288,4,0.246,147.486,11,best sad songs,1
96,Calum Scott,False,Dancing On My Own,Only Human (Deluxe),79,0.8370,0.681,4,0.174,0.000034,...,0.0983,-8.745,major,0.0315,4,0.231,112.672,11,best sad songs,1
97,Sam Smith,False,"Fire On Fire - From ""Watership Down""","Fire On Fire (From ""Watership Down"")",67,0.4760,0.584,4,0.407,0.000000,...,0.1800,-7.356,minor,0.0460,4,0.341,115.124,11,best sad songs,1
98,Billie Eilish,False,bury a friend,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",74,0.7400,0.905,3,0.389,0.162000,...,0.1060,-14.505,major,0.3320,4,0.196,120.046,11,best sad songs,1
